veri seti: https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-Convolutional-Neural-Networks.zip

1000 köpek 1000 kedi fotoğrafı var

## 1. Kütüphaneler

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

Using TensorFlow backend.


In [2]:
# Sequential: yapay sinir ağı oluşturmak için
# Convolution2D: ANN öncesi convolutional aşamasında ilk katman. 2D foto olduğu
#                için videoda zaman boyutu da var
# MaxPooling2D: CNN aşamasında pooling aşaması için
# Flatten: CNN aşaması sonunda ANN'e hazırlık için. Fully connecter layer'a 
#           input olabilmesi için
# Dense: ANN katmanları eklemek için

![image.png](cnn_phases.png)
Görsel: https://www.udemy.com/deeplearning

![image.png](cnn_architecture.png)

Görsel: Peng, Min, et al. Nirfacenet: A convolutional neural network for near-infrared face identification. Information 7.4 (2016): 61.

## 2. YAPAY SİNİR AĞI NESNESİ OLUŞTURMA
    Yapay sinir ağı iki farklı şekilde başlatılabilir: 
    1. Katmanlar dizilimi (Sequantial layers) olarak. 
    2. Graph olarak.
    Birinci yolda sadece yukarıda indirilen Sequantial sınıfından nesne yaratılır. 
    Bu da Karar ağacı, SVM gibi bir sınıflandırıcı olacak.

In [3]:
classifier = Sequential()

W0717 07:32:31.218311 14592 deprecation_wrapper.py:119] From c:\users\user\envs\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



## 3. CONVOLUTION LAYER

![image.gif](cnn_animation.gif)

Görsel: https://hackernoon.com/visualizing-parts-of-convolutional-neural-networks-using-keras-and-cats-5cc01b214e59

In [4]:
classifier.add(
    Convolution2D(filters=32, 
                  kernel_size=(3,3), 
                  input_shape=(64,64,3),
                  activation='relu')
              )

W0717 07:32:31.316001 14592 deprecation_wrapper.py:119] From c:\users\user\envs\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 07:32:31.318008 14592 deprecation_wrapper.py:119] From c:\users\user\envs\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



## 4. POOLING STEP

![image.gif](cnn_max_pooling.gif)

Görsel: https://www.oreilly.com/ideas/visualizing-convolutional-neural-networks

In [5]:
# temel işlevi ann'e girecek vektör boyutunu indirmektir. Bununla birlikte bilgikaybı 
# yaşatmamaya çalışır.

In [6]:
classifier.add(MaxPooling2D(pool_size= (2, 2) ))

W0717 07:32:32.138811 14592 deprecation_wrapper.py:119] From c:\users\user\envs\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



## 5. FLATTENING STEP

In [7]:
classifier.add(Flatten())

## 6. FULL CONNECTION STEP

In [8]:
# Bu aşamadan sonra klasik bir ann

In [9]:
# Input layer cnn tarafından geldiği için koymuyoruz. 
# output (hidden layer) boyutu konusunda belli bir kural yok
# output: Zayıf öğrenme olmayacak kadar büyük, hesaplamayı orlaştırmayacak kadar küçük
classifier.add(Dense(units = 128, activation = 'relu'))

## 7. OUTPUT LAYER

In [10]:
classifier.add(Dense(units = 1, activation = 'sigmoid'))

## 8. COMPILING

In [11]:
# optimizer: Stochastic Gradient Descent'i temsilen adam
# loss: SGD'ni optimizasyonu için kullanılacak loss function. 
#       Tahmin y ile gerçek ye değeri arasını hesaplayıp en optimal değeri
#        SGD'ye buldurur. Logaritmik loss function. Binary olduğu için 
#       binary_crossentropy kullanıyoruz.
# metrics: İlave olarak burada model değerlendirme kriterleri belirlenir. Bir liste halinde verilir. 
#          Biz şimdilik sadece accuracy veriyoruz.

In [12]:
classifier.compile(optimizer= 'adam', loss= 'binary_crossentropy',
                  metrics=['accuracy'])

W0717 07:32:34.273486 14592 deprecation_wrapper.py:119] From c:\users\user\envs\tensorflow-1\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0717 07:32:34.289433 14592 deprecation_wrapper.py:119] From c:\users\user\envs\tensorflow-1\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0717 07:32:34.294451 14592 deprecation.py:323] From c:\users\user\envs\tensorflow-1\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## 9. CNN MODELİNİ EĞİTME

In [13]:
# Keras ImagePreprocessing
# Augmentation overfitting'i azaltır.
# flow from directory

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'D:/Datasets/ignored_data/cnn_cats_dogs/dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'D:/Datasets/ignored_data/cnn_cats_dogs/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

classifier.fit_generator(
        training_set,
        steps_per_epoch=8000,
        epochs=3,
        validation_data=test_set,
        validation_steps=2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/3
8000/8000 [==============================] - 776s 97ms/step - loss: 0.3477 - acc: 0.8426 - val_loss: 0.8149 - val_acc: 0.7611
Epoch 2/3
8000/8000 [==============================] - 758s 95ms/step - loss: 0.1340 - acc: 0.9485 - val_loss: 1.4437 - val_acc: 0.7279
Epoch 3/3
8000/8000 [==============================] - 760s 95ms/step - loss: 0.0765 - acc: 0.9724 - val_loss: 1.5515 - val_acc: 0.7529
